In [2]:
import os
os.chdir("..")

In [3]:
import os

os.environ["CONFIG_APP_DIR"] = "config"
import json
import re

In [4]:
import random

random.randint(1, 100000)

1020

In [5]:
from etl_pipeline.service.proto.api.etl_pipeline_pb2_grpc import EtlPipelineServiceStub
from etl_pipeline.config import ConsulServiceConfig
from etl_pipeline.service.proto.api.etl_pipeline_pb2 import (
    FAILURE,
    SUCCESS,
    Alert,
    Match,
    RunEtlRequest,
)


def load_alert(filepath: str = "notebooks/sample/wm_address_in_payload_format.json"):
    with open(filepath, "r") as f:
        text = json.load(f)
        ids = [
            text[i]
            for key in text
            for i in re.compile(r".*matchRecords\[\d+\].matchId.*").findall(key)
            if i
        ]
        matches_ids = [
            Match(match_id=i, match_name=f"alerts/2/matches/{random.randint(1, 100000)}") for num, i in enumerate(ids)
        ]
        alert = Alert(batch_id="1", alert_name="alerts/2", matches=matches_ids)
        for key, value in text.items():
            alert.flat_payload[str(key)] = str(value)
    return alert

In [6]:
request_alert = load_alert("notebooks/sample/big_fat_flat_payload.json")

In [7]:
import grpc
service_config = ConsulServiceConfig()
with open(service_config.GRPC_CLIENT_TLS_CA, "rb") as f:
    ca = f.read()
with open(service_config.GRPC_CLIENT_TLS_PRIVATE_KEY, "rb") as f:
    private_key = f.read()
with open(service_config.GRPC_CLIENT_TLS_PUBLIC_KEY_CHAIN, "rb") as f:
    certificate_chain = f.read()
server_credentials = grpc.ssl_channel_credentials(ca, private_key, certificate_chain)
channel = grpc.insecure_channel("localhost:9090")

In [83]:
import grpc
# channel = grpc.insecure_channel("localhost:9090",  options=[
#             ("grpc.max_send_message_length", 27838500),
#             ("grpc.max_receive_message_length", 27838500),
#         ])
stub = EtlPipelineServiceStub(channel)
from time import time
start= time()
%timeit response = stub.RunEtl(RunEtlRequest(alerts=[load_alert("notebooks/sample/big_fat_flat_payload.json") for i in range(100)]), timeout=60)
time() - start

5.72 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


45.54953122138977

In [80]:
response.etl_alerts[-1].etl_status

1

In [28]:
from itertools import zip_longest
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [29]:
list(zip([1,2], [3,4], grouper([1,2], 2)))

[(1, 3, (1, 2))]

In [11]:
def load_alert(filepath: str = "notebooks/sample/wm_address_in_payload_format.json"):
    with open(filepath, "r") as f:
        text = json.load(f)
        import pdb; pdb.set_trace()
        matches = [Match(match_id="0", match_name=f"alerts/2/matches/{num}")for num, _ in enumerate([re.findall(r"matchRecords\[\d+\]", key)[0] for key in text if re.search(r"matchRecords\[\d+\]", key)])]
        alert = Alert(batch_id="1", alert_name="alerts/2", matches=matches)
        for key, value in text.items():
            alert.flat_payload[str(key)] = str(value)
    return alert


In [13]:
with open("notebooks/sample/wm_address_in_payload_format.json", "r") as f:
        text = json.load(f)
        

In [14]:
import re

['49159509', '49159506']

In [16]:
matches

[match_id: "0"
 match_name: "alerts/2/matches/0",
 match_id: "0"
 match_name: "alerts/2/matches/1"]